In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, InputLayer, Lambda, Flatten, Conv2D, Input, Conv1D, Reshape, Conv1DTranspose, MaxPool1D
from tensorflow.keras.optimizers import Adam

# Importing Data

In [2]:
raw_data = pd.read_csv(r"C:\Users\singh\OneDrive\Desktop\Work\ACRi Internship Work\DataSet\Shock_3_Solutions.csv")
raw_data = np.array(raw_data, dtype='float')

#normalising the data
raw_data[:,0] = (raw_data[:,0]-np.min(raw_data[:,0]))/(1.1*(np.max(raw_data[:,0])-np.min(raw_data[:,0])))
raw_data[:,1] = (raw_data[:,1]-np.min(raw_data[:,1]))/(1.1*(np.max(raw_data[:,1])-np.min(raw_data[:,1])))

#number of inputs
n_ip = int(raw_data.shape[0])

In [4]:
print(n_ip)

6006


# Encoder and Decoder

In [20]:
input1 = Input(shape=(int(raw_data.shape[1]), int(raw_data.shape[0]), ));
EC1 = Conv1D(filters=1,kernel_size=3,activation='tanh')(input1)
EC2 = MaxPool1D(pool_size=1, strides=1)(EC1)
EC3 = Conv1D(1,kernel_size=3,activation='tanh')(EC2)
EC4 = MaxPool1D(pool_size=1, strides=1)(EC3)
EC5 = Conv1D(1,kernel_size=3,activation='tanh')(EC4)
EC6 = MaxPool1D(pool_size=3, strides=1)(EC5)
EC = Flatten(EC6)
ED = Dense(units = 50, activation="tanh", kernel_initializer="glorot_normal")(EC)
Encoder = Model(inputs = input1, outputs = ED)

#decoder
input2 = Input(shape=(50,));
DC1 =  Dense(units = 150, activation="tanh", kernel_initializer="glorot_normal")(input2)
DC2 = Reshape((50, 3))(DC1)
DC3 = Conv1DTranspose(3, 100, activation='tanh')(DC2)
DC4 = Conv1DTranspose(3, 200, activation='tanh')(DC3)
DC = Conv1DTranspose(3, n_ip, activation='tanh')(DC4)
Decoder = Model(inputs = input2, outputs = DC)

#Autoencoder
IP = Input(shape=(n_ip, 3,))
BN = Encoder(inp)
OP = Decoder(BN)
autoencoder = Model(inputs = IP, outputs = OP)

ValueError: Negative dimension size caused by subtracting 3 from 1 for '{{node conv1d_10/conv1d}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_10/conv1d/ExpandDims, conv1d_10/conv1d/ExpandDims_1)' with input shapes: [?,1,1,1], [1,3,1,1].

In [ ]:
Encoder.summary()
keras.utils.plot_model(Encoder)

In [ ]:
Decoder.summary()
keras.utils.plot_model(Decoder)

In [ ]:
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(raw_data, raw_data, epochs=20)